In [4]:
import git
from pathlib import Path
import os

ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

FINAL_DATA_NAME = 'approx1e5-agriVision-fourier' # + channel
CONSTANT_SAMPLE_SIZE = int(1e5)
RAW_DATA_SUFFIX = "agriVision-RGB-cleaned"

data_dir = os.path.join(ROOT_DIR, 'raw-data','agriVision')
file_list = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(data_dir)
data_dir

'c:\\Users\\yashd\\Desktop\\hierarchical-bayesian-model-validation\\raw-data\\agriVision'

In [5]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from transform import *
os.chdir(os.path.join(ROOT_DIR, "dataset-preparation"))

In [6]:
file_list = [os.path.join(data_dir, f"full-{RAW_DATA_SUFFIX}", filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(os.path.join(data_dir, f"full-{RAW_DATA_SUFFIX}"))
file_names[:5]

['batch 1', 'batch 2', 'batch 3']

In [7]:
'''Assuming No batching is required. Not applicable for agriVision'''

# data_dir = os.path.join(ROOT_DIR, "raw-data", "agriVision", "full-agriVision-RGB-cleaned")

# for channel in ['red', 'blue', 'green', 'gray', 'infrared']:

#     channel_fr = convert_to_fourier_basis(data_dir, channel, debug = True)
#     pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"full-agriVision-fourier-{channel}-df.pickle"))

#     min_group, max_group = 2, max(channel_fr['band'])
#     group_data_map = dict()
#     group_data_map_size = dict()
#     for group in np.arange(min_group, max_group + 1):
#         data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
#         group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
#         group_data_map_size[group] = data.size
    
#     pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
#     pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
    

'Assuming No batching is required. Not applicable for agriVision'

In [8]:
'''To split large dataset into many batches, only needs to be run once'''
# k = 10000
# target_dir = os.path.join(ROOT_DIR, 'raw-data', 'agriVision') # Where the batch{i} folders will be created
# directorySplit(folder_dir = data_dir, target_dir = target_dir, name = RAW_DATA_SUFFIX, k = k)
# print(f"{len(file_names)//k} batches created" )

'To split large dataset into many batches, only needs to be run once'

In [9]:
'''Show all subsets of data in raw data folder that have already been created'''
print(''.join([x+"\n" for x in os.listdir(data_dir) if x.__contains__(RAW_DATA_SUFFIX)]))
BATCH_NUM = 0

batch0-agriVision-RGB-cleaned
full-agriVision-RGB-cleaned
toy-agriVision-RGB-cleaned



# Fourier

In [10]:
#Values obtained from plots in agriVisionFourierEDA.ipynb
batch_dir = os.path.join(ROOT_DIR, "raw-data", "agriVision", f"batch{BATCH_NUM}-{RAW_DATA_SUFFIX}")
splits = getSplits(0.008052940675034493,0.5282905274067067, 1.15)
print(splits)

[0.008052940675034493, 0.009260881776289667, 0.010650014042733115, 0.012247516149143082, 0.014084643571514543, 0.016197340107241723, 0.01862694112332798, 0.021420982291827175, 0.02463412963560125, 0.028329249080941435, 0.03257863644308265, 0.037465431909545044, 0.0430852466959768, 0.04954803370037331, 0.056980238755429305, 0.0655272745687437, 0.07535636575405526, 0.08665982061716354, 0.09965879370973807, 0.11460761276619877, 0.13179875468112856, 0.15156856788329784, 0.1743038530657925, 0.20044943102566135, 0.23051684567951053, 0.2650943725314371, 0.30485852841115263, 0.3505873076728255, 0.4031754038237493, 0.4636517143973116]


In [11]:
channel = "red"

channel_fr = convert_to_fourier_basis(batch_dir, channel, split_list = splits, debug = True, image_opener = npz_opener)
channel_fr['data'] = channel_fr['data'].apply(lambda x : x.astype(np.float32))
pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))


min_group, max_group = 2, max(channel_fr['band'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_fr.head()

100%|██████████| 4500/4500 [06:15<00:00, 12.00it/s]


[0.00805294 0.00926088 0.01065001 0.01224752 0.01408464 0.01619734
 0.01862694 0.02142098 0.02463413 0.02832925 0.03257864 0.03746543
 0.04308525 0.04954803 0.05698024 0.06552727 0.07535637 0.08665982
 0.09965879 0.11460761 0.13179875 0.15156857 0.17430385 0.20044943
 0.23051685 0.26509437 0.30485853 0.35058731 0.4031754  0.46365171]


100%|██████████| 30/30 [01:13<00:00,  2.44s/it]


,band,channel,magnitude_endpoints,unique_magnitudes,data
0,1,red,"(0.0, 0.0078125)",10,"[-118092.0, -209566.81, 80184.59, -10856.434, ..."
1,2,red,"(0.008052940675034493, 0.008734640537108554)",3,"[-2029.8997, -180.56264, 2404.6543, -2978.5793..."
2,3,red,"(0.009765625, 0.010517900013934578)",3,"[7238.685, 8688.75, -69.16031, -8039.517, -219..."
3,4,red,"(0.011048543456039806, 0.01188039556698871)",4,"[9876.31, 5580.1226, -927.84576, 1846.173, 292..."
4,5,red,"(0.012352647110032733, 0.014084184669781208)",6,"[-289.08902, -4747.2026, 10580.036, 2368.381, ..."


In [ ]:
channel = "blue"

channel_fr = convert_to_fourier_basis(batch_dir, channel, split_list = splits, debug = True, image_opener = npz_opener)
channel_fr['data'] = channel_fr['data'].apply(lambda x : x.astype(np.float32))
pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))


min_group, max_group = 2, max(channel_fr['band'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_fr.head()

100%|██████████| 4500/4500 [10:49<00:00,  6.93it/s]


[0.00805294 0.00926088 0.01065001 0.01224752 0.01408464 0.01619734
 0.01862694 0.02142098 0.02463413 0.02832925 0.03257864 0.03746543
 0.04308525 0.04954803 0.05698024 0.06552727 0.07535637 0.08665982
 0.09965879 0.11460761 0.13179875 0.15156857 0.17430385 0.20044943
 0.23051685 0.26509437 0.30485853 0.35058731 0.4031754  0.46365171]


100%|██████████| 30/30 [00:56<00:00,  1.90s/it]


,band,channel,magnitude_endpoints,unique_magnitudes,data
0,1,blue,"(0.0, 0.0078125)",10,"[-170415.45814724237, 510926.1970722508, 12243..."
1,2,blue,"(0.008052940675034493, 0.008734640537108554)",3,"[804.2062570760761, -3518.844141769729, 5547.6..."
2,3,blue,"(0.009765625, 0.010517900013934578)",3,"[1635.7426808248724, -4306.874994654427, -5431..."
3,4,blue,"(0.011048543456039806, 0.01188039556698871)",4,"[1375.6718053559239, 4996.1382836703415, 7108...."
4,5,blue,"(0.012352647110032733, 0.014084184669781208)",6,"[-94.99611658441336, -2011.3474954596052, -205..."


In [ ]:
channel = "green"

channel_fr = convert_to_fourier_basis(batch_dir, channel, split_list = splits, debug = True, image_opener = npz_opener)
channel_fr['data'] = channel_fr['data'].apply(lambda x : x.astype(np.float32))
pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_fr['band'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_fr.head()

100%|██████████| 4500/4500 [05:39<00:00, 13.27it/s]


[0.00805294 0.00926088 0.01065001 0.01224752 0.01408464 0.01619734
 0.01862694 0.02142098 0.02463413 0.02832925 0.03257864 0.03746543
 0.04308525 0.04954803 0.05698024 0.06552727 0.07535637 0.08665982
 0.09965879 0.11460761 0.13179875 0.15156857 0.17430385 0.20044943
 0.23051685 0.26509437 0.30485853 0.35058731 0.4031754  0.46365171]


100%|██████████| 30/30 [01:04<00:00,  2.15s/it]


,band,channel,magnitude_endpoints,unique_magnitudes,data
0,1,green,"(0.0, 0.0078125)",10,"[-148763.5560309001, 211995.89178617718, 12813..."
1,2,green,"(0.008052940675034493, 0.008734640537108554)",3,"[-89.69857697371067, 34.62541306817903, 3640.0..."
2,3,green,"(0.009765625, 0.010517900013934578)",3,"[982.6646464557805, -19958.29065489971, -2931...."
3,4,green,"(0.011048543456039806, 0.01188039556698871)",4,"[710.7463532448172, 10861.789584987408, 10518...."
4,5,green,"(0.012352647110032733, 0.014084184669781208)",6,"[-307.97392063466515, 4390.759568037944, -3727..."


In [ ]:
channel = "gray"

channel_fr = convert_to_fourier_basis(batch_dir, channel, split_list = splits, debug = True, image_opener = npz_opener)
channel_fr['data'] = channel_fr['data'].apply(lambda x : x.astype(np.float32))
pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))


min_group, max_group = 2, max(channel_fr['band'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_fr.head()

100%|██████████| 4500/4500 [07:02<00:00, 10.65it/s]


[0.00805294 0.00926088 0.01065001 0.01224752 0.01408464 0.01619734
 0.01862694 0.02142098 0.02463413 0.02832925 0.03257864 0.03746543
 0.04308525 0.04954803 0.05698024 0.06552727 0.07535637 0.08665982
 0.09965879 0.11460761 0.13179875 0.15156857 0.17430385 0.20044943
 0.23051685 0.26509437 0.30485853 0.35058731 0.4031754  0.46365171]


100%|██████████| 30/30 [01:19<00:00,  2.64s/it]


,band,channel,magnitude_endpoints,unique_magnitudes,data
0,1,gray,"(0.0, 0.0078125)",10,"[-142049.2683456862, 120047.65561441513, 11314..."
1,2,gray,"(0.008052940675034493, 0.008734640537108554)",3,"[-567.7105609121869, -434.7932870428433, 3487...."
2,3,gray,"(0.009765625, 0.010517900013934578)",3,"[2926.941804880599, -9609.433048423809, -2360...."
3,4,gray,"(0.011048543456039806, 0.01188039556698871)",4,"[3526.0636900539075, 8613.32885122238, 6707.49..."
4,5,gray,"(0.012352647110032733, 0.014084184669781208)",6,"[-278.01895837392277, 929.1434425598386, -2105..."


In [ ]:
channel = "infrared"

channel_fr = convert_to_fourier_basis(batch_dir, channel, split_list = splits, debug = True, image_opener = npz_opener)
channel_fr['data'] = channel_fr['data'].apply(lambda x : x.astype(np.float32))
pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))


min_group, max_group = 2, max(channel_fr['band'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_fr.head()

100%|██████████| 4500/4500 [05:42<00:00, 13.12it/s]


[0.00805294 0.00926088 0.01065001 0.01224752 0.01408464 0.01619734
 0.01862694 0.02142098 0.02463413 0.02832925 0.03257864 0.03746543
 0.04308525 0.04954803 0.05698024 0.06552727 0.07535637 0.08665982
 0.09965879 0.11460761 0.13179875 0.15156857 0.17430385 0.20044943
 0.23051685 0.26509437 0.30485853 0.35058731 0.4031754  0.46365171]


100%|██████████| 30/30 [00:40<00:00,  1.36s/it]


,band,channel,magnitude_endpoints,unique_magnitudes,data
0,1,infrared,"(0.0, 0.0078125)",10,"[-142049.2683456862, 120047.65561441513, 11314..."
1,2,infrared,"(0.008052940675034493, 0.008734640537108554)",3,"[-567.7105609121869, -434.7932870428433, 3487...."
2,3,infrared,"(0.009765625, 0.010517900013934578)",3,"[2926.941804880599, -9609.433048423809, -2360...."
3,4,infrared,"(0.011048543456039806, 0.01188039556698871)",4,"[3526.0636900539075, 8613.32885122238, 6707.49..."
4,5,infrared,"(0.012352647110032733, 0.014084184669781208)",6,"[-278.01895837392277, 929.1434425598386, -2105..."


In [ ]:
del channel_fr, group_data_map, group_data_map_size

# Wavelet

In [8]:
FINAL_DATA_NAME = 'approx1e5-agriVision-wavelet'
batch_dir = os.path.join(ROOT_DIR, "raw-data", "agriVision", f"batch{BATCH_NUM}-{RAW_DATA_SUFFIX}")

In [ ]:
channel = "red"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float16))
pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0]).astype(int)
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"test_int_batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

In [ ]:
channel = "red"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float16))
pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0])
    group_data_map[group] = np.sort(np.round(data, 2))[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"test_batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

10 layers being used


100%|██████████| 4500/4500 [41:14<00:00,  1.82it/s]  


MemoryError: Unable to allocate 2.20 GiB for an array with shape (294912000,) and data type float64

In [ ]:
channel = "green"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float16))
pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0])
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

10 layers being used


  7%|▋         | 295/4500 [00:23<05:33, 12.63it/s]


KeyboardInterrupt: 

In [ ]:
channel = "blue"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float16))
pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))


min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0])
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

10 layers being used


,channel,layer,orientation,data
0,Green,1,L1,"[159.79101562500017, 913.0214843750011, 440.91..."
1,Green,2,D,"[-54.748046875000064, 60.771484375000185, 6.72..."
2,Green,2,H,"[64.33007812500007, 52.45507812500003, 16.5664..."
3,Green,2,V,"[-90.8027343750001, 0.33007812499994316, -6.49..."
4,Green,3,D,"[9.527343750000007, 5.507812500000005, -2.4296..."


In [ ]:
channel = "gray"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float16))
pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))


min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0])
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

In [ ]:
channel = "infrared"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float16))
pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))


min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0])
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

10 layers being used


,channel,layer,orientation,data
0,Blue,1,L1,"[190.46875000000023, 903.0976562500011, 474.31..."
1,Blue,2,D,"[-79.18359375000007, 59.93750000000008, -0.888..."
2,Blue,2,H,"[90.8710937500001, 49.05078124999999, -14.1269..."
3,Blue,2,V,"[-121.25781250000013, -5.492187500000057, -8.1..."
4,Blue,3,D,"[8.69140625000001, 37.976562500000036, -2.8476..."
